In [1]:
import flwr as fl
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from numpy import newaxis
import math
import os
import pandas as pd
import torch.nn as nn
from scipy.stats import chi2
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Function
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from centralized import DomainAdaptationModel,ReviewDataset
from sklearn.model_selection import train_test_split

c:\Users\hl100\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class DomainAdaptationClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, test_loader, device):
        self.model = model
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.device = device

    def get_parameters(self):
        return [val.cpu().numpy() for val in self.model.state_dict().values()]

    def set_parameters(self, parameters):
        state_dict = {k: torch.tensor(v) for k, v in zip(self.model.state_dict().keys(), parameters)}
        self.model.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.train()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
        for epoch in range(1):  # Perform a single epoch of training
            for batch in self.train_loader:
                input_ids, attention_mask, token_type_ids, labels = [x.to(self.device) for x in batch]
                sentiment_pred, domain_pred = self.model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
                loss = self.compute_loss(sentiment_pred, domain_pred, labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        return self.get_parameters(), len(self.train_loader.dataset), {}

    def compute_loss(self, sentiment_pred, domain_pred, labels):
        sentiment_loss = torch.nn.CrossEntropyLoss()(sentiment_pred, labels)
        domain_loss = torch.nn.CrossEntropyLoss()(domain_pred, labels)  # Assuming domain labels are in labels
        return sentiment_loss + domain_loss

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in self.test_loader:
                input_ids, attention_mask, token_type_ids, labels = [x.to(self.device) for x in batch]
                sentiment_pred, domain_pred = self.model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
                _, predicted = torch.max(sentiment_pred, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        return float(accuracy), len(self.test_loader.dataset), {}
    




In [4]:
torch.cuda.is_available()

False

In [8]:
#Model
model = DomainAdaptationModel()
tokenizer = AutoTokenizer.from_pretrained('jackaduma/SecBERT')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)
state_dict_path = 'C:/Users/hl100/Downloads/' + 'size250k_1epoch_1_model.bin'
model.load_state_dict(torch.load(state_dict_path, map_location=device))
#Data
df_full = pd.read_csv('D:\Hoc\SecBert\SecBERT\multilabel-train\dataset_capec.csv')
df_full['text'] = df_full['text'].str.replace('/',' ')
df_train = df_full.groupby('label').head(2000)
# df_train = df_full
df_train = df_train.dropna(subset=['label'])
label_counts = df_train['label'].value_counts()
print(label_counts)
print(df_train.size)


train_df, test_df = train_test_split(df_train, test_size=0.2, random_state=42)
train_texts = train_df['text'].values
train_labels = train_df['label'].values
test_texts = test_df['text'].values
test_labels = test_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(df_train['text'], df_train['label'],test_size=0.3, stratify=df_train['label'], shuffle = True)
df_train = pd.concat([X_train, Y_train], axis=1)
df_test = pd.concat([X_test, Y_test], axis=1)

# Tokenize the loaded texts for training and testing
train_dataset = ReviewDataset(df_train)
test_dataset = ReviewDataset(df_test)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False) 


<>:10: SyntaxWarning: invalid escape sequence '\H'
<>:10: SyntaxWarning: invalid escape sequence '\H'
C:\Users\hl100\AppData\Local\Temp\ipykernel_18224\140660674.py:10: SyntaxWarning: invalid escape sequence '\H'
  df_full = pd.read_csv('D:\Hoc\SecBert\SecBERT\multilabel-train\dataset_capec.csv')
c:\Users\hl100\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


cpu


C:\Users\hl100\AppData\Local\Temp\ipykernel_18224\140660674.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(state_dict_path, map_locatio

label
000 - Normal                              2000
126 - Path Traversal                      2000
66 - SQL Injection                        2000
272 - Protocol Manipulation               2000
310 - Scanning for Vulnerable Software    2000
242 - Code Injection                      2000
194 - Fake the Source of Data             2000
34 - HTTP Response Splitting              2000
153 - Input Data Manipulation             1387
Name: count, dtype: int64
34774


c:\Users\hl100\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
# Simulate clients
client = DomainAdaptationClient(model, train_loader, test_loader, device)
fl.client.start_numpy_client(server_address="localhost:8080", client=client)

	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
	)
	Using `start_numpy_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.
 -- 10061)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.\r\n -- 10061)", grpc_status:14, created_time:"2024-09-09T10:46:15.465458+00:00"}"
>